In [162]:
# setup e test librerie
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

evaluation

overfitting

Importare tutto

In [163]:
mushrooms = pd.read_csv("mushrooms_cleaned.csv")

y = mushrooms["is-edible"]
X_all = mushrooms.drop(columns="is-edible")

Select KBest features and only use those

In [164]:
from sklearn.feature_selection import SelectKBest

selector = SelectKBest()
best_features = selector.fit_transform(X_all, y)

X = pd.DataFrame(best_features, columns=selector.get_feature_names_out())

Splitting data

In [165]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=1/3,
    random_state=42
)

Creating function for printing an evaluation

In [166]:
def print_eval(X, y, model):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print(f"Accuracy/Score: {accuracy:.5}")
    print("Classification report")
    print(classification_report(y, y_pred))

Creating function for printing the coefficients of the features

In [167]:
def print_coefficients(classifier):
    print("Coefficients")
    for i in range(len(X.columns)):
        print(f"{X.columns[i]}: {classifier.coef_[0][i]}")

Try and evaluate logistic regression

In [168]:
from sklearn.linear_model import LogisticRegression

model = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(solver="saga"))
])
model.fit(X_train, y_train)

# print_coefficients(model.named_steps["lr"])
print_eval(X_val, y_val, model)

Accuracy/Score: 0.77447
Classification report
              precision    recall  f1-score   support

       False       0.81      0.84      0.83      3933
        True       0.70      0.65      0.67      2186

    accuracy                           0.77      6119
   macro avg       0.76      0.75      0.75      6119
weighted avg       0.77      0.77      0.77      6119



Try and evaluate decision tree

In [169]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

model = Pipeline([
    ("scaler", StandardScaler()),
    ("dtc", DecisionTreeClassifier(max_depth=3))
])
model.fit(X_train, y_train)

# plot_tree(model.named_steps["dtc"])
print_eval(X_val, y_val, model)

Accuracy/Score: 0.77153
Classification report
              precision    recall  f1-score   support

       False       0.86      0.77      0.81      3933
        True       0.65      0.77      0.71      2186

    accuracy                           0.77      6119
   macro avg       0.76      0.77      0.76      6119
weighted avg       0.78      0.77      0.77      6119



Try and evaluate ridge

In [170]:
from sklearn.linear_model import RidgeClassifier

model = Pipeline([
    ("scaler", StandardScaler()),
    ("rc", RidgeClassifier(alpha=0.5))
])
model.fit(X_train, y_train)

# print_coefficients(model.named_steps["rc"])
print_eval(X_val, y_val, model)

Accuracy/Score: 0.77153
Classification report
              precision    recall  f1-score   support

       False       0.82      0.83      0.82      3933
        True       0.69      0.66      0.67      2186

    accuracy                           0.77      6119
   macro avg       0.75      0.75      0.75      6119
weighted avg       0.77      0.77      0.77      6119



Try and evaluate Random forest

In [171]:
from sklearn.ensemble import RandomForestClassifier

model = Pipeline([
    ("scaler", StandardScaler()),
    ("rfc", RandomForestClassifier())
])
model.fit(X_train, y_train)

print_eval(X_val, y_val, model)

Accuracy/Score: 1.0
Classification report
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3933
        True       1.00      1.00      1.00      2186

    accuracy                           1.00      6119
   macro avg       1.00      1.00      1.00      6119
weighted avg       1.00      1.00      1.00      6119



Try and evaluate svm

In [172]:
from sklearn.svm import SVC

model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(random_state=42))
])
model.fit(X_train, y_train)

print_eval(X_val, y_val, model)

Accuracy/Score: 0.99967
Classification report
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3933
        True       1.00      1.00      1.00      2186

    accuracy                           1.00      6119
   macro avg       1.00      1.00      1.00      6119
weighted avg       1.00      1.00      1.00      6119



KNN classifier

In [173]:
from sklearn.neighbors import KNeighborsClassifier

model = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])
model.fit(X_train, y_train)

print_eval(X_val, y_val, model)

Accuracy/Score: 1.0
Classification report
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3933
        True       1.00      1.00      1.00      2186

    accuracy                           1.00      6119
   macro avg       1.00      1.00      1.00      6119
weighted avg       1.00      1.00      1.00      6119

